In [2]:
!(pip3 install duckdb)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 20.7 MB/s eta 0:00:00a 0:00:01


In [1]:
import duckdb

In [7]:
!(pip3 install pandas)

  Using cached pandas-2.2.3-cp310-cp310-macosx_11_0_arm64.whl.metadata (89 kB)
Using cached pandas-2.2.3-cp310-cp310-macosx_11_0_arm64.whl (11.3 MB)


In [2]:
con = duckdb.connect('mimic.db')

# List all tables
print(con.execute("SHOW TABLES").fetchdf())

         name
0  ADMISSIONS
1    DRGCODES
2  D_ICDPROCS
3    ICUSTAYS
4    PATIENTS
5   PROCS_ICD


In [3]:
# Query admissions table
con.sql("SELECT hadm_id, deathtime FROM ADMISSIONS")

┌─────────┬─────────────────────┐
│ hadm_id │      deathtime      │
│  int32  │       varchar       │
├─────────┼─────────────────────┤
│  142345 │ NULL                │
│  105331 │ 2126-08-28 18:59:00 │
│  165520 │ 2125-10-07 15:13:00 │
│  199207 │ NULL                │
│  177759 │ 2163-05-15 12:00:00 │
│  103770 │ NULL                │
│  199395 │ NULL                │
│  132349 │ NULL                │
│  140372 │ NULL                │
│  157235 │ NULL                │
│     ·   │  ·                  │
│     ·   │  ·                  │
│     ·   │  ·                  │
│  167021 │ NULL                │
│  167612 │ 2152-10-09 22:33:00 │
│  110958 │ NULL                │
│  125157 │ NULL                │
│  131048 │ NULL                │
│  198330 │ NULL                │
│  174245 │ 2178-05-15 09:45:00 │
│  163189 │ NULL                │
│  192189 │ NULL                │
│  103379 │ NULL                │
├─────────┴─────────────────────┤
│      129 rows (20 shown)      │
└─────────────

In [4]:
con.sql("SELECT SUM(expire_flag) FROM PATIENTS")

┌──────────────────┐
│ sum(expire_flag) │
│      int128      │
├──────────────────┤
│              100 │
└──────────────────┘

In [5]:
con.sql("SHOW TABLES")

┌────────────┐
│    name    │
│  varchar   │
├────────────┤
│ ADMISSIONS │
│ DRGCODES   │
│ D_ICDPROCS │
│ ICUSTAYS   │
│ PATIENTS   │
│ PROCS_ICD  │
└────────────┘

In [6]:
# Get the list of tables
tables = con.execute("SHOW TABLES").fetchall()

# Loop through each table and count rows
for table_name, in tables:
    count = con.execute(f"SELECT COUNT(*) FROM {table_name}").fetchone()[0]
    print(f"Table '{table_name}' has {count} rows.")

Table 'ADMISSIONS' has 129 rows.
Table 'DRGCODES' has 297 rows.
Table 'D_ICDPROCS' has 3882 rows.
Table 'ICUSTAYS' has 136 rows.
Table 'PATIENTS' has 100 rows.
Table 'PROCS_ICD' has 506 rows.


In [7]:
con.sql("""
		SELECT count(*) FROM PATIENTS
		WHERE gender = 'F'
		""")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│           55 │
└──────────────┘

In [8]:
con.sql("""
		SELECT count(*) FROM PATIENTS
		WHERE dod_hosp IS NOT NULL
		""")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│           70 │
└──────────────┘

In [9]:
con.sql("""
		SELECT count(*) FROM ADMISSIONS
		WHERE deathtime IS NOT NULL
		""")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│           40 │
└──────────────┘

In [11]:
con.sql("""
		SELECT DISTINCT admission_type FROM ADMISSIONS
		""")

┌────────────────┐
│ admission_type │
│    varchar     │
├────────────────┤
│ ELECTIVE       │
│ EMERGENCY      │
│ URGENT         │
└────────────────┘

In [15]:
con.sql("""
	SELECT * FROM ICUSTAYS
	JOIN ADMISSIONS ON ICUSTAYS.hadm_id = ADMISSIONS.hadm_id
""")

┌────────┬────────────┬─────────┬────────────┬────────────┬────────────────┬───────────────┬──────────────┬─────────────┬─────────────────────┬─────────────────────┬─────────┬────────┬────────────┬─────────┬─────────────────────┬─────────────────────┬─────────────────────┬────────────────┬───────────────────────────┬──────────────────────────┬───────────┬──────────┬───────────────────┬────────────────┬────────────────────────┬─────────────────────┬─────────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬──────────────────────┬──────────────────────┐
│ row_id │ subject_id │ hadm_id │ icustay_id │  dbsource  │ first_careunit │ last_careunit │ first_wardid │ last_wardid │       intime        │       outtime       │   los   │ row_id │ subject_id │ hadm_id │      admittime      │      dischtime      │      deathtime      │ admission_type │    admission_location     │    discharge_location    │ insurance │ la

In [13]:
con.sql("""
    WITH DRG_ADM_COUNTS AS (
        SELECT 
            drg_code, 
            COUNT(DISTINCT hadm_id) AS num_admissions
        FROM DRGCODES
        GROUP BY drg_code
    )
    SELECT * FROM DRG_ADM_COUNTS
    ORDER BY num_admissions DESC
""")


┌──────────┬────────────────┐
│ drg_code │ num_admissions │
│ varchar  │     int64      │
├──────────┼────────────────┤
│ 7204     │             12 │
│ 871      │             11 │
│ 416      │              8 │
│ 1394     │              4 │
│ 7104     │              4 │
│ 208      │              3 │
│ 1304     │              3 │
│ 207      │              3 │
│ 004      │              2 │
│ 578      │              2 │
│  ·       │              · │
│  ·       │              · │
│  ·       │              · │
│ 470      │              1 │
│ 6914     │              1 │
│ 020      │              1 │
│ 3013     │              1 │
│ 320      │              1 │
│ 657      │              1 │
│ 0454     │              1 │
│ 2833     │              1 │
│ 105      │              1 │
│ 089      │              1 │
├──────────┴────────────────┤
│    162 rows (20 shown)    │
└───────────────────────────┘

In [ ]:
# Calculate the age of each patient at the time of admission.
# Hint: cast(patients.dob as date) allows for addition and subtraction of dates (in days)

# Identify if a patient passes away when they are in the ICU.

# Calculate the average duration of admission.

In [28]:
con.sql("""
    WITH AD_PAT AS (
        SELECT *, 
               DATE_DIFF('day', CAST(dob AS DATE), CAST(admittime AS DATE)) / 365.25 AS age_at_admission
        FROM PATIENTS
        JOIN ADMISSIONS ON PATIENTS.subject_id = ADMISSIONS.subject_id
    )
    SELECT subject_id, age_at_admission FROM AD_PAT
""")


┌────────────┬────────────────────┐
│ subject_id │  age_at_admission  │
│   int32    │       double       │
├────────────┼────────────────────┤
│      10006 │  70.63381245722108 │
│      10011 │  36.18891170431211 │
│      10013 │  87.08281998631075 │
│      10017 │  73.67556468172485 │
│      10019 │  48.89801505817933 │
│      10026 │ 299.99452429842574 │
│      10027 │  82.49144421629022 │
│      10029 │  78.44763860369609 │
│      10032 │   88.0082135523614 │
│      10033 │  81.62354551676934 │
│        ·   │          ·         │
│        ·   │          ·         │
│        ·   │          ·         │
│      43881 │  53.58521560574949 │
│      43909 │  78.87748117727584 │
│      43927 │  71.63586584531143 │
│      44083 │  54.46406570841889 │
│      44083 │  54.51334702258727 │
│      44083 │  54.52977412731006 │
│      44154 │ 299.99452429842574 │
│      44212 │ 45.437371663244356 │
│      44222 │  73.06228610540725 │
│      44228 │  58.14647501711157 │
├────────────┴──────────────

In [35]:
con.sql("""
    WITH ICUS AS (
        SELECT *,
		dischtime = deathtime AS died_in_ICU
        FROM ICUSTAYS
        JOIN ADMISSIONS ON ICUSTAYS.hadm_id = ADMISSIONS.hadm_id
    )
    SELECT * FROM ICUS
""")


┌────────┬────────────┬─────────┬────────────┬────────────┬────────────────┬───────────────┬──────────────┬─────────────┬─────────────────────┬─────────────────────┬─────────┬──────────┬──────────────┬───────────┬─────────────────────┬─────────────────────┬─────────────────────┬────────────────┬───────────────────────────┬──────────────────────────┬───────────┬──────────┬───────────────────┬────────────────┬────────────────────────┬─────────────────────┬─────────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬──────────────────────┬──────────────────────┬─────────────┐
│ row_id │ subject_id │ hadm_id │ icustay_id │  dbsource  │ first_careunit │ last_careunit │ first_wardid │ last_wardid │       intime        │       outtime       │   los   │ row_id_1 │ subject_id_1 │ hadm_id_1 │      admittime      │      dischtime      │      deathtime      │ admission_type │    admission_location     │    discharge_lo

In [41]:
con.sql("""
    WITH ADMIT_DUR AS (
        SELECT *, 
               ROUND(DATE_DIFF('hour', CAST(admittime AS TIMESTAMP), CAST(dischtime AS TIMESTAMP))/24,2) AS duration_stay
        FROM ADMISSIONS
        JOIN PATIENTS ON PATIENTS.subject_id = ADMISSIONS.subject_id
    )
    SELECT subject_id, duration_stay FROM ADMIT_DUR
""")


┌────────────┬───────────────┐
│ subject_id │ duration_stay │
│   int32    │    double     │
├────────────┼───────────────┤
│      10006 │          8.83 │
│      10011 │         13.83 │
│      10013 │          2.67 │
│      10017 │          8.04 │
│      10019 │          0.67 │
│      10026 │          7.17 │
│      10027 │         12.29 │
│      10029 │         10.17 │
│      10032 │         12.79 │
│      10033 │          3.54 │
│        ·   │            ·  │
│        ·   │            ·  │
│        ·   │            ·  │
│      43881 │          8.08 │
│      43909 │          0.13 │
│      43927 │          4.13 │
│      44083 │          7.25 │
│      44083 │          2.92 │
│      44083 │         10.04 │
│      44154 │          0.54 │
│      44212 │          36.0 │
│      44222 │          1.29 │
│      44228 │          9.63 │
├────────────┴───────────────┤
│    129 rows (20 shown)     │
└────────────────────────────┘

In [43]:
conn = duckdb.connect('mimic_new.db', read_only = False)

conn.sql('SHOW TABLES;')

┌─────────┐
│  name   │
│ varchar │
├─────────┤
│ 0 rows  │
└─────────┘

In [44]:
import pandas as pd
df = pd.read_csv('penguins.csv')
df.to_sql("PENGUINS", conn, index = False)
conn.close()

FileNotFoundError: [Errno 2] No such file or directory: 'penguins.csv'